In [1]:

train_msg = open("../../MCMD/java/sort_random_train80_valid10_test10/train.msg.txt", encoding='UTF-8').read().split("\n")
len(train_msg)

In [7]:
valid_msg = open("../../MCMD/java/sort_random_train80_valid10_test10/valid.msg.txt", encoding='UTF-8').read().split("\n")
len(valid_msg)

45001

In [9]:
test_msg = open("../../MCMD/java/sort_random_train80_valid10_test10/test.msg.txt", encoding='UTF-8').read().split("\n")
len(test_msg)

45001

## Preprocess script

In [4]:
import re
def process_diff(diff_text):
    # Replace <nl> with \n
    diff_text = diff_text.replace('<nl>', '\n')
    diff_text = re.sub(r'(?<=\S)\s*([^\w\s])\s*(?=\S)', r'\1', diff_text)
    diff_text = re.sub(r'\n\s+', r'\n', diff_text)
    return diff_text

def process_msg(msg_text):
    msg_text = re.sub(r'(?<=\S)\s*([^\w\s])\s*(?=\S)', r'\1', msg_text)
    return msg_text

In [5]:
import json

selected_messages = []
selected_languages = []
selected_diffs = []
nmt_msg = []
nngen_msg = []

def extract_diffs(indices, diff_file_path, nngen_path, nmt_path):
    with open(diff_path, 'r', encoding='UTF-8') as diff_file:
        diff_line = diff_file.readlines()
    
    with open(nngen_path, 'r', encoding='UTF-8') as nngen_file:
        nngen_line = nngen_file.readlines()

    with open(nmt_path, 'r', encoding='UTF-8') as nmt_file:
        nmt_line = nmt_file.readlines()
    
    
    for i in indices:
        diff = process_diff(diff_line[i])
        nngen = nngen_line[i]
        nmt = nmt_line[i]
        selected_diffs.append(diff)
        nngen_msg.append(nngen)
        nmt_msg.append(nmt)

mcmd_base_path = "../../MCMD"
models_base_path = "./data"

languages = ["java", "cpp", "csharp", "python", "javascript"]
file_names = ["test.msg.txt", "test.diff.txt"]

test_msg_path = [f"{mcmd_base_path}/{lang}/sort_random_train80_valid10_test10/test.msg.txt" for lang in languages]
test_diff_path = [f"{mcmd_base_path}/{lang}/sort_random_train80_valid10_test10/test.diff.txt" for lang in languages]

nngen_msg_path = [f"{models_base_path}/model_NNGen/data_MCMD/{lang}/split_random/gen.msg" for lang in languages]
nmt_msg_path = [f"{models_base_path}/model_NMT/data_MCMD/{lang}/split_random/gen.msg" for lang in languages]


for msg_path, diff_path, nngen_path, nmt_path in zip(test_msg_path, test_diff_path, nngen_msg_path, nmt_msg_path):
    selected_indices = []
    with open(msg_path, 'r', encoding='UTF-8') as f:
        count = 0
        for i, line in enumerate(f):
            if count>= 1000:
                break
            # line = process_msg(line)
            if len(line.split()) >= 5:
                selected_indices.append(i)
                selected_messages.append(line.strip())
                selected_languages.append(msg_path.split('/')[3])
                count += 1
            
        extract_diffs(selected_indices, diff_path, nngen_path, nmt_path)

# Create a list of dictionaries containing selected messages and diffs
data = []
for msg, diff, nngen, nmt, language in zip(selected_messages, selected_diffs, nngen_msg, nmt_msg, selected_languages):
    item = {
        'msg': msg,
        'diff': diff,
        'nngen': nngen,
        'nmt': nmt,
        'language': language
    }
    data.append(item)

# Write the data to a JSON file
with open('data/selected_data.json', 'w', encoding='UTF-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)


In [20]:
import torch
from transformers import RobertaTokenizer, RobertaConfig, RobertaForCausalLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base", cache_dir='./')
model = RobertaForCausalLM.from_pretrained("microsoft/codebert-base", cache_dir='./', is_decoder=True)
model.to(device)

Some weights of RobertaForCausalLM were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForCausalLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): 

In [23]:
def generate_description(code):
    inputs = tokenizer(code, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_length=128, num_return_sequences=1)
    description = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return description

In [25]:
generate_description("The following is a diff which describes the code changes in a commit, Your task is to write a short commit message accordingly. diff --git a/src/gwt/src/org/rstudio/studio/client/workbench/views/source/editors/text/TextEditingTargetRenameHelper.java b/src/gwt/src/org/rstudio/studio/client/workbench/views/source/editors/text/TextEditingTargetRenameHelper.java\nindex e8d986667..ce243ca1f 100644\n--- a/src/gwt/src/org/rstudio/studio/client/workbench/views/source/editors/text/TextEditingTargetRenameHelper.java\n+++ b/src/gwt/src/org/rstudio/studio/client/workbench/views/source/editors/text/TextEditingTargetRenameHelper.java\n@@ -44,7 +44,7 @@ public class TextEditingTargetRenameHelper\n       \n       editor_.setCursorPosition(position);\n       \n-      // Validate that we're looking at an R identifier (TODO: refactor strings?)\n+      // Validate that we're looking at an R identifier\n       String targetValue = cursor.currentValue();\n       String targetType = cursor.currentType();\n       \n@@ -104,7 +104,6 @@ public class TextEditingTargetRenameHelper\n       }\n       \n       // Otherwise, just rename the variable within the current scope.\n-      // TODO: Do we need to look into parent scopes?\n       return renameVariablesInScope(\n             editor_.getCurrentScope(),\n             targetValue,\nAccording to the diff, the commit message should be:")

C:\Users\tyfann\anaconda3\envs\phi\lib\site-packages\transformers\generation\utils.py:1363: UserWarning: Input length of input_ids is 488, but `max_length` is set to 128. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


"The following is a diff which describes the code changes in a commit, Your task is to write a short commit message accordingly. diff --git a/src/gwt/src/org/rstudio/studio/client/workbench/views/source/editors/text/TextEditingTargetRenameHelper.java b/src/gwt/src/org/rstudio/studio/client/workbench/views/source/editors/text/TextEditingTargetRenameHelper.java\nindex e8d986667..ce243ca1f 100644\n--- a/src/gwt/src/org/rstudio/studio/client/workbench/views/source/editors/text/TextEditingTargetRenameHelper.java\n+++ b/src/gwt/src/org/rstudio/studio/client/workbench/views/source/editors/text/TextEditingTargetRenameHelper.java\n@@ -44,7 +44,7 @@ public class TextEditingTargetRenameHelper\n       \n       editor_.setCursorPosition(position);\n       \n-      // Validate that we're looking at an R identifier (TODO: refactor strings?)\n+      // Validate that we're looking at an R identifier\n       String targetValue = cursor.currentValue();\n       String targetType = cursor.currentType();\n 